## Prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

import statsmodels.api as sm
from scipy import stats
import statsmodels.tools as tools
from scipy.stats import bootstrap

from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 100
pd.options.display.max_columns = None

# Set a seed for NumPy
np.random.seed(0)

# Set a seed for Python's built-in random module
random.seed(0)

### Raw data

In [2]:
mesa_raw = pd.read_csv('../data_processed/MESA/Y_BaselineX_raw_full.csv')

diet_phys_map = {0:1, 1:2, 2:3}
mesa_raw['nutrition'] = mesa_raw['nutrition'].replace(diet_phys_map)
mesa_raw['PhysAct'] = mesa_raw['PhysAct'].replace(diet_phys_map)

mesa_raw = mesa_raw.dropna()

# standardize continuous variables
cols_to_standardize = ['nSES','nFavFood','nPhysFac','nRS', 'sbp', 'hdl', 'totchol']
scaler = StandardScaler()
mesa_raw[cols_to_standardize] = scaler.fit_transform(mesa_raw[cols_to_standardize])


mesa_bla_raw = mesa_raw[mesa_raw['race'] == 3]

In [ ]:
jhs_outcome_full = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')
jhs_outcome = jhs_outcome_full[['subjid','event','time']].copy()

jhs_covar = pd.read_csv('../jhs/data/processed/jhs_raw_full.csv')
jhs_covar = jhs_covar[jhs_covar['visit'] == 1]

jhs_raw = pd.merge(jhs_outcome, jhs_covar, on=['subjid'], how='left')

jhs_raw = jhs_raw.drop(jhs_raw.columns[4], axis=1)

jhs_raw['nutrition'] = jhs_raw['nutrition'].replace(diet_phys_map)
jhs_raw['PhysAct'] = jhs_raw['PhysAct'].replace(diet_phys_map)

jhs_impute = jhs_raw.copy()

jhs_raw = jhs_raw.dropna()

# standardize continuous variables
cols_to_standardize = ['nSES','nFavFood','nPhysFac','nRS', 'sbp', 'hdl', 'totchol']
scaler = StandardScaler()
jhs_raw[cols_to_standardize] = scaler.fit_transform(jhs_raw[cols_to_standardize])

### Raw data with missing imputed 

In [ ]:
mesa_imputed = pd.read_csv('../data_processed/MESA/Y_BaselineX_imputed_full.csv')

diet_phys_map = {0:1, 1:2, 2:3}
mesa_imputed['nutrition'] = mesa_imputed['nutrition'].replace(diet_phys_map)
mesa_imputed['PhysAct'] = mesa_imputed['PhysAct'].replace(diet_phys_map)

# standardize continuous variables
cols_to_standardize = ['nSES','nFavFood','nPhysFac','nRS', 'sbp', 'hdl', 'totchol']
scaler = StandardScaler()
mesa_imputed[cols_to_standardize] = scaler.fit_transform(mesa_imputed[cols_to_standardize])

mesa_bla_imputed = mesa_imputed[mesa_imputed['race'] == 3]

In [ ]:
# jhs

### Raw data with categorization

In [ ]:
def quantile_exp(df,con_exp_feat):
    for feat in con_exp_feat:
        df[feat] = df[feat].transform(lambda x: pd.qcut(x.rank(method='first'), 
                                                         q = [0, 0.25, 0.5, 0.75, 1], labels = [1,2,3,4]))
        df[feat] = pd.to_numeric(df[feat])
    return df

con_exp_feat = ['nSES','nFavFood','nPhysFac','nRS']

In [ ]:
# read from MESA_raw

mesa_cate = mesa_raw.copy()
mesa_cate = quantile_exp(mesa_cate,con_exp_feat)

mesa_bla_cate = mesa_bla_raw.copy()
mesa_bla_cate = quantile_exp(mesa_bla_cate,con_exp_feat)

### Processed data

In [ ]:
mesa_pre = pd.read_csv('../data_processed/MESA/Y_BaselineX_processed_full.csv')

diet_phys_map = {0:1, 1:2, 2:3}
mesa_pre['nutrition'] = mesa_pre['nutrition'].replace(diet_phys_map)
mesa_pre['PhysAct'] = mesa_pre['PhysAct'].replace(diet_phys_map)

mesa_bla_pre = mesa_pre[mesa_pre['race'] == 3]

In [ ]:
jhs_pre = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')
jhs_pre = jhs_pre.drop("nSES", axis=1)

jhs_pre['nutrition'] = jhs_pre['nutrition'].replace(diet_phys_map)
jhs_pre['PhysAct'] = jhs_pre['PhysAct'].replace(diet_phys_map)

## Moderation test

In [ ]:
# continuous variable standardized
# ordinal categorical and treat as continuous variable
# bootstrap to report CI for the interaction coefficient 

In [ ]:
named_df_list = [('mesa_bla_raw', mesa_bla_raw), ('mesa_bla_imputed', mesa_bla_imputed), ('mesa_bla_cate', mesa_bla_cate),
                 ('mesa_bla_pre', mesa_bla_pre)]
Y_var = 'Y_10y'
X_var = ['nSES','nFavFood','nPhysFac', 'nRS',
         'FamIncome','nutrition', 'PhysAct','currentSmoker','alc',
         'age','gender','race','Diabetes','hdl','totchol','sbp']

report_df = pd.DataFrame(columns=['Data','Bootstrap Result Name', 'Mean', '2.5%', '97.5%'])

for df_name, df in named_df_list:

    ### estimate CI using bootstrapping
    
    # empty lists to store results
    num_bootstrap_samples = 1000
    
    bp_1_nSES_FamIncome = []
    bp_2_nFavFood_FamIncome = []
    bp_3_nPhysFac_FamIncome = []
    bp_4_nRS_FamIncome = []
    
    bp_5_nSES_nutrition = []
    bp_6_nFavFood_nutrition = []
    bp_7_nPhysFac_nutrition = []
    bp_8_nRS_nutrition = []
    
    bp_9_nSES_PhysAct = []
    bp_10_nFavFood_PhysAct = []
    bp_11_nPhysFac_PhysAct = []
    bp_12_nRS_PhysAct = []
    
    bp_13_nSES_currentSmoker = []
    bp_14_nFavFood_currentSmoker = []
    bp_15_nPhysFac_currentSmoker = []
    bp_16_nRS_currentSmoker = []
    
    bp_17_nSES_alc = []
    bp_18_nFavFood_alc = []
    bp_19_nPhysFac_alc = []
    bp_20_nRS_alc = []
    
    bootstrap_results_all = [bp_1_nSES_FamIncome,bp_2_nFavFood_FamIncome,bp_3_nPhysFac_FamIncome,bp_4_nRS_FamIncome,
                             bp_5_nSES_nutrition,bp_6_nFavFood_nutrition,bp_7_nPhysFac_nutrition,bp_8_nRS_nutrition,
                            bp_9_nSES_PhysAct,bp_10_nFavFood_PhysAct,bp_11_nPhysFac_PhysAct,bp_12_nRS_PhysAct,
                            bp_13_nSES_currentSmoker,bp_14_nFavFood_currentSmoker,bp_15_nPhysFac_currentSmoker,bp_16_nRS_currentSmoker,
                            bp_17_nSES_alc,bp_18_nFavFood_alc,bp_19_nPhysFac_alc,bp_20_nRS_alc]
    
    bootstrap_names = ['bp_1_nSES_FamIncome','bp_2_nFavFood_FamIncome','bp_3_nPhysFac_FamIncome','bp_4_nRS_FamIncome',
                       'bp_5_nSES_nutrition','bp_6_nFavFood_nutrition','bp_7_nPhysFac_nutrition','bp_8_nRS_nutrition',
                       'bp_9_nSES_PhysAct','bp_10_nFavFood_PhysAct','bp_11_nPhysFac_PhysAct','bp_12_nRS_PhysAct',
                       'bp_13_nSES_currentSmoker','bp_14_nFavFood_currentSmoker','bp_15_nPhysFac_currentSmoker','bp_16_nRS_currentSmoker',
                       'bp_17_nSES_alc','bp_18_nFavFood_alc','bp_19_nPhysFac_alc','bp_20_nRS_alc']

    for _ in range(num_bootstrap_samples):
        
        #print(_)
        
        # Draw a bootstrap sample
        bootstrap_sample = df.sample(frac=1, replace=True)
        
        # extract X and Y
        y = bootstrap_sample['Y_10y']
        X = bootstrap_sample[X_var]
        
        # dummy coding
        X = pd.get_dummies(X, columns=['gender','race','Diabetes'], 
                    drop_first=True)
        
        # add interaction term to separate df
        X1 = X.copy()
        X2 = X.copy()
        X3 = X.copy()
        X4 = X.copy()
        X5 = X.copy()
        X6 = X.copy()
        X7 = X.copy()
        X8 = X.copy()
        X9 = X.copy()
        X10 = X.copy()
        X11 = X.copy()
        X12 = X.copy()
        X13 = X.copy()
        X14 = X.copy()
        X15 = X.copy()
        X16 = X.copy()
        X17 = X.copy()
        X18 = X.copy()
        X19 = X.copy()
        X20 = X.copy()

        X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
        X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
        X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
        X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
        
        X5['nSES_nutrition'] = X['nSES'] * X['nutrition']
        X6['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
        X7['nPhysFac_nutrition'] = X['nPhysFac'] * X['nutrition']
        X8['nRS_nutrition'] = X['nRS'] * X['nutrition']
        
        X9['nSES_PhysAct'] = X['nSES'] * X['PhysAct']
        X10['nFavFood_PhysAct'] = X['nFavFood'] * X['PhysAct']
        X11['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']
        X12['nRS_PhysAct'] = X['nRS'] * X['PhysAct']
        
        X13['nSES_currentSmoker'] = X['nSES'] * X['currentSmoker']
        X14['nFavFood_currentSmoker'] = X['nFavFood'] * X['currentSmoker']
        X15['nPhysFac_currentSmoker'] = X['nPhysFac'] * X['currentSmoker']
        X16['nRS_currentSmoker'] = X['nRS'] * X['currentSmoker']
        
        X17['nSES_alc'] = X['nSES'] * X['alc']
        X18['nFavFood_alc'] = X['nFavFood'] * X['alc']
        X19['nPhysFac_alc'] = X['nPhysFac'] * X['alc']
        X20['nRS_alc'] = X['nRS'] * X['alc']
        
        
        # fit regression and store results respectively
        dfs = [X1, X2, X3, X4, X5, X6, X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20]
        
        for i, df_test in enumerate(dfs):
            logistic_model_with_moderation = sm.Logit(y, sm.add_constant(df_test)).fit(disp=0)
            last_param = logistic_model_with_moderation.params[-1]
            bootstrap_results_all[i].append(last_param)
        

    ### report result
    
    print(f"Moderation results in dataset: {df_name}\n")

    for i, bootstrap_result in enumerate(bootstrap_results_all):
        # Calculate required statistics
        mean = np.mean(bootstrap_result)
        percentile_2_5 = round(np.percentile(bootstrap_result, 2.5), 5)
        percentile_97_5 = round(np.percentile(bootstrap_result, 97.5), 5)

        # Add to the DataFrame
        report_df = report_df.append({
            'Data':df_name,
            'Bootstrap Result Name': bootstrap_names[i],
            'Mean': mean,
            '2.5%': percentile_2_5,
            '97.5%': percentile_97_5
        }, ignore_index=True)

    # Print the DataFrame
    print(report_df)

    print("\n" + "="*50 + "\n")  

    
export_path = f"../results/MESA_Bla/moderation.csv"
report_df.to_csv(export_path)

## Mediation Test

In [ ]:
named_df_list = [('mesa_raw', mesa_raw), ('mesa_imputed', mesa_imputed), 
                 ('mesa_cate', mesa_cate), ('mesa_pre', mesa_pre)]

Y = 'Y_10y'
X = 'nPhysFac'
M = 'PhysAct'
Z = ['nSES','nFavFood','nRS',
      'FamIncome','nutrition','currentSmoker','alc',
    'age','gender','race','Diabetes','hdl','totchol','sbp']

In [ ]:
### Y = b0 + b1X + bZ + e (Z represents all the other covariates)
### M = b0 + b2X + e
### Y = b0 + b3X + b4M + bZ + e

report_df = pd.DataFrame(columns = ['Data','Effect','Estimate','95_CI_Lower','95_CI_Upper'])


for df_name, df in named_df_list:

    ### estimate CI using bootstrapping

    num_bootstrap_samples = 1000
    bootstrap_result_b1 = []
    bootstrap_result_b2 = []
    bootstrap_result_b3 = []
    bootstrap_result_b4 = []

    for _ in range(num_bootstrap_samples):
        # Draw bootstrap sample
        bootstrap_sample = df.sample(frac=1, replace=True)

        # Y ~ X + Z
        model_1 = sm.Logit(bootstrap_sample[Y], sm.add_constant(bootstrap_sample[[X]+Z])).fit(disp=0)  ## +Z
        bootstrap_result_b1.append(model_1.params[X])
        
        # M ~ X
        model_2 = sm.OLS(bootstrap_sample[M], sm.add_constant(bootstrap_sample[X])).fit()
        bootstrap_result_b2.append(model_2.params[X])

        # Y ~ X + M +Z
        model_3 = sm.Logit(bootstrap_sample[Y], sm.add_constant(bootstrap_sample[[X]+[M]+Z])).fit(disp=0)
        bootstrap_result_b3.append(model_3.params[X])
        bootstrap_result_b4.append(model_3.params[M])
        

    
    ### report result

    print(f"Mediation results using: {df_name}\n")
    print(f"Neighborhood exposure variable: {X}\n")
    print(f"Individual level potential mediator tested: {M}\n")
    
    # result for a single dataset
    new_df = pd.DataFrame(
        {'Data': [df_name, df_name,df_name,df_name],
         'Effect': ['ACME', 'ADE', 'Total Effect', 'Prop. Mediated'], 
         'Estimate': [round((np.mean(bootstrap_result_b2)*np.mean(bootstrap_result_b3)), 5),
                      round(np.mean(bootstrap_result_b3), 5),
                      round(np.mean(bootstrap_result_b1), 5),
                      round(np.mean(bootstrap_result_b2)*np.mean(bootstrap_result_b3)/np.mean(bootstrap_result_b1), 5)],
         '95_CI_Lower': [round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3), 2.5),5),
                         round(np.percentile(bootstrap_result_b3, 2.5),5),
                         round(np.percentile(bootstrap_result_b1, 2.5),5),
                         round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3)/np.array(bootstrap_result_b1), 2.5),5)],
         '95_CI_Upper': [round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3), 97.5),5),
                         round(np.percentile(bootstrap_result_b3, 97.5),5),
                         round(np.percentile(bootstrap_result_b1, 97.5),5),
                         round(np.percentile(np.array(bootstrap_result_b2)*np.array(bootstrap_result_b3)/np.array(bootstrap_result_b1), 97.5),5)]
        })
    
    # append to the complete dataset
    report_df = pd.concat([report_df, new_df], ignore_index=True)
    
    # print(report_df)
    
    print("\n" + "="*50 + "\n")  

export_path = f"../results/MESA/mediation/mediation_{X}_{M}.csv"
report_df.to_csv(export_path)